## Work
1. 請改變 reduce_lr 的 patience 和 factor 並比較不同設定下，對訓練/驗證集的影響
2. 請將 optimizer 換成 Adam、RMSprop 搭配 reduce_lr 並比較訓練結果

In [ ]:
import os
import keras
import itertools
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD,Adam,RMSprop 
from keras.callbacks import ReduceLROnPlateau

# Disable GPU
#os.environ["CUDA_VISIBLE_DEVICES"] = ""
#force to use cpu(by changing device_count in the tf.ContigProto)
import tensorflow as tf
from keras import backend as K
num_cores = 4
config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,
                        inter_op_parallelism_threads=num_cores, 
                        allow_soft_placement=True,
                        device_count = {'CPU' : 1,
                                        'GPU' : 0}
                       )

session = tf.Session(config=config)
K.set_session(session)

In [ ]:
train, test = keras.datasets.cifar10.load_data()

In [ ]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [ ]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [ ]:
# write function to build model
def build_mlp():
    """
    Build your own model
    """
    model=Sequential()
    model.add(Dense(units=input_shape[1], input_dim=input_shape[1],kernel_initializer='normal',activation='relu',name='input'))
    for i, n_units in enumerate(num_neurons):
        model.add(Dense(units=n_units,kernel_initializer='normal',activation='relu',name='hidden'+str(i+1)))
    model.add(Dense(units=output_units,kernel_initializer='normal',activation='softmax',name='output'))
    return model

In [ ]:
# write function to built optimizer
def build_opt(opt):
    if opt =='SGD':
        return SGD(lr=LEARNING_RATE,nesterov=True, momentum=MOMENTUM)
    elif opt=='Adam':
        return Adam(lr=LEARNING_RATE,beta_1=0.9,beta_2=0.999,epsilon=None,decay=0.0,amsgrad=False)
    elif opt=='RMSprop':
        return RMSprop(lr=LEARNING_RATE,rho=0.9,epsilon=None,decay=0.0)

In [ ]:
# write function to plot results
def plot_results(results):
    color_bar=['dodgerblue','greenyellow','red','sienna','lime','green','aqua','maroon','mediumblue','slateblue','darkorchid','purple']
    """
    Draw the results
    """
    plt.figure(figsize=(12,9))
    for i, cond in enumerate(results.keys()):
        plt.plot(range(len(results[cond]['train_loss'])), results[cond]['train_loss'], '-', label='train'+cond, color=color_bar[i%12])
        plt.plot(range(len(results[cond]['valid_loss'])), results[cond]['valid_loss'],'--', label='valid'+cond, color=color_bar[i%12])
    plt.title('Loss')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show

    plt.figure(figsize=(12,9))
    for i, cond in enumerate(results.keys()):
        plt.plot(range(len(results[cond]['train_acc'])), results[cond]['train_acc'], '-', label='train'+cond, color=color_bar[i%12])
        plt.plot(range(len(results[cond]['valid_acc'])), results[cond]['valid_acc'],'--', label='valid'+cond, color=color_bar[i%12])
    plt.title('Accuracy')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

In [ ]:
## 超參數設定
"""
Set your hyper-parameters
"""
LEARNING_RATE=1e-3
EPOCHS=30
BATCH_SIZE=256
MOMENTUM=0.9
REDUCE_LR_PATIENCE=[5,8]
REDUCE_LR_FACTOR=[0.5,0.8]
OPTIMIZERS=['SGD','Adam','RMSprop']

## 作業一、改變 reduce_lr 的 patience 和 factor 並比較不同設定下，對訓練/驗證集的影響

In [ ]:
# 載入 Callbacks
"""
Write your training loop and show the results
"""
results_D87_hw1={}
for p, f in itertools.product(REDUCE_LR_PATIENCE,REDUCE_LR_FACTOR):
    reduce_lr = ReduceLROnPlateau(factor=f, 
                                  min_lr=1e-12, 
                                  monitor='val_loss', 
                                  patience=p, 
                                  verbose=1)
    model = build_mlp(input_shape=x_train.shape)
    model.summary()
    optimizer = build_opt('SGD')
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True,
              callbacks=[reduce_lr]
             )

    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    #make dictionary for results
    exp_name_tag= f"[p={p}, f={f}]"
    results_D87_hw1[exp_name_tag]={'train_loss':train_loss,
                                   'valid_loss':valid_loss,
                                   'train_acc':train_acc,
                                   'valid_acc':valid_acc}

In [ ]:
# 繪製結果
"""
plot the results
"""
plot_results(results_D87_hw1)

## 作業二、將 optimizer 換成 Adam、RMSprop 搭配 reduce_lr 並比較訓練結果

In [ ]:
results_D87_hw2={}
for opt,p,f in itertools.product(OPTIMIZERS,REDUCE_LR_PATIENCE,REDUCE_LR_FACTOR):
    reduce_lr = ReduceLROnPlateau(factor=f, 
                                  min_lr=1e-12, 
                                  monitor='val_loss', 
                                  patience=p, 
                                  verbose=1)
    model = build_mlp(input_shape=x_train.shape)
    model.summary()
    optimizer = build_opt(opt)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True,
              callbacks=[reduce_lr]
             )

    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    #make dictionary for results
    exp_name_tag= f"{opt}[p={p}, f={f}]"
    results_D87_hw2[exp_name_tag]={'train_loss':train_loss,
                                   'valid_loss':valid_loss,
                                   'train_acc':train_acc,
                                   'valid_acc':valid_acc}

In [ ]:
plot_results(results_D87_hw2)